In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

/Users/thomassullivan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Example of scraping 2024 stats

In [48]:
# url of 2024 all pro team
url = "https://www.pro-football-reference.com/years/2024/allpro.htm"
response = requests.get(url)

# initiate BeautifulSoup html parser
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'id': 'all_pro'}) # find the appropriate table
header_row = table.find_all('tr')[0] # header row is first row of table
column_headers = [th.text.strip() for th in header_row.find_all('th')] # strip for specific column title
data_rows = table.find_all('tr')[1:] # data for individual rows

data = []

# iterate through data rows and append data to the list
for row in data_rows:
    pos = row.find('th').text.strip()  # Extract the position from the <th> element
    row_data = [td.text.strip() for td in row.find_all('td')] # all other data points in that row
    row_data.insert(0, pos) # add position as first position of list
    data.append(row_data) 

# create a Pandas DataFrame from list
df = pd.DataFrame(data, columns=column_headers[:])
# rename duplicate column titles
df.columns.values[8] = "PsAtt"
df.columns.values[9] = "PsYds"
df.columns.values[10] = "PsTD"
df.columns.values[11] = "PsInt"
df.columns.values[13] = "RshYds"
df.columns.values[14] = "RshTD"
df.columns.values[16] = "RecYds"
df.columns.values[17] = "RecTD"
df.fillna(0, inplace=True) # na occurs when a player has 0 years of experience
df.head(10)

,Pos,Player,Tm,Age,Yrs,G,GS,Cmp,PsAtt,PsYds,...,Att,RshYds,RshTD,Rec,RecYds,RecTD,Solo,Sk,Int,All-pro teams
0,QB,Lamar Jackson,BAL,27,6,17,17,316,474,4172,...,139,915,4,0,0,0,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
1,QB,Jared Goff,DET,30,8,17,17,390,539,4629,...,35,56,0,1,7,1,0,0.0,0,FW: 1st Tm All-Conf.
2,QB,Josh Allen,BUF,28,6,17,17,307,483,3731,...,102,531,12,0,7,1,0,0.0,0,AP: 2nd Tm
3,QB,Joe Burrow,CIN,28,4,17,17,460,652,4918,...,42,201,2,0,0,0,0,0.0,0,PFF: 2nd Tm
4,RB,Saquon Barkley,PHI,27,6,16,16,0,0,0,...,345,2005,13,33,278,2,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
5,RB,Derrick Henry,BAL,30,8,17,17,0,0,0,...,325,1921,16,19,193,2,0,0.0,0,"FW: 1st Tm All-Conf., AP: 2nd Tm, FW: 1st Tm, ..."
6,RB,Jahmyr Gibbs,DET,22,1,17,4,0,0,0,...,250,1412,16,52,517,4,0,0.0,0,FW: 1st Tm All-Conf.
7,FB,Kyle Juszczyk,SFO,33,11,17,15,0,0,0,...,5,26,1,19,200,2,0,0.0,0,"AP: 2nd Tm, NFLPA: 1st Tm"
8,FB,Patrick Ricard,BAL,30,7,17,6,0,0,0,...,0,0,0,3,22,1,0,0.0,0,AP: 1st Tm
9,WR,Justin Jefferson,MIN,25,4,17,17,1,1,22,...,1,3,0,103,1533,10,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pos            90 non-null     object 
 1   Player         90 non-null     object 
 2   Tm             90 non-null     object 
 3   Age            90 non-null     int64  
 4   Yrs            87 non-null     float64
 5   G              90 non-null     int64  
 6   GS             90 non-null     int64  
 7   PsAtt          90 non-null     int64  
 8   PsYds          90 non-null     int64  
 9   PsTD           90 non-null     int64  
 10  PsInt          90 non-null     int64  
 11  Int            90 non-null     object 
 12  RshYds         90 non-null     object 
 13  RshTD          90 non-null     object 
 14  TD             90 non-null     object 
 15  RecYds         90 non-null     object 
 16  RecTD          90 non-null     object 
 17  TD             90 non-null     object 
 18  Solo        

#### clean up column data types

In [49]:
cat_cols = ["Pos", "Player", "Tm", "All-pro teams"] # not converted to ints
num_cols = [col for col in df.columns if col not in cat_cols]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pos            90 non-null     object 
 1   Player         90 non-null     object 
 2   Tm             90 non-null     object 
 3   Age            90 non-null     int64  
 4   Yrs            87 non-null     float64
 5   G              90 non-null     int64  
 6   GS             90 non-null     int64  
 7   Cmp            90 non-null     int64  
 8   PsAtt          90 non-null     int64  
 9   PsYds          90 non-null     int64  
 10  PsTD           90 non-null     int64  
 11  PsInt          90 non-null     int64  
 12  Att            90 non-null     int64  
 13  RshYds         90 non-null     int64  
 14  RshTD          90 non-null     int64  
 15  Rec            90 non-null     int64  
 16  RecYds         90 non-null     int64  
 17  RecTD          90 non-null     int64  
 18  Solo        

In [29]:
df.tail()

,Player,Tm,Age,Yrs,G,GS,Cmp,PsAtt,PsYds,PsTD,...,Att,RshYds,RshTD,Rec,RecYds,RecTD,Solo,Sk,Int,All-pro teams
85,Derius Davis,LAC,24,1.0,15,2,0,0,0,0,...,12,39,0,13,112,2,0,0.0,0,FW: 1st Tm All-Conf.
86,Austin Ekeler,WAS,29,7.0,12,6,0,0,0,0,...,77,367,4,35,366,0,0,0.0,0,AP: 2nd Tm
87,Brenden Schooler,NWE,27,2.0,17,0,0,0,0,0,...,0,0,0,0,0,0,9,2.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
88,J.T. Gray,NOR,28,6.0,17,0,0,0,0,0,...,0,0,0,0,0,0,12,0.0,0,"FW: 1st Tm All-Conf., AP: 2nd Tm, PFF: 1st Tm"
89,Miles Killebrew,PIT,31,8.0,17,0,0,1,0,0,...,0,0,0,0,0,0,8,0.0,0,NFLPA: 1st Tm


### scrape all-pro data from last 25 years

In [2]:
def year_scrape(yr):
    url = f"https://www.pro-football-reference.com/years/{yr}/allpro.htm"
    response = requests.get(url)

    # initiate BeautifulSoup html parser
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'all_pro'}) # find the appropriate table
    header_row = table.find_all('tr')[0] # header row is first row of table
    column_headers = [th.text.strip() for th in header_row.find_all('th')] # strip for specific column title
    data_rows = table.find_all('tr')[1:] # data for individual rows

    data = []

    # iterate through data rows and append data to the list
    for row in data_rows:
        pos = row.find('th').text.strip()  # Extract the position from the <th> element
        row_data = [td.text.strip() for td in row.find_all('td')] # all other data points in that row
        row_data.insert(0, pos) # add position as first position of list
        data.append(row_data) 

    # create a Pandas DataFrame from list
    df = pd.DataFrame(data, columns=column_headers[:])
    # rename duplicate column titles
    df.columns.values[8] = "PsAtt"
    df.columns.values[9] = "PsYds"
    df.columns.values[10] = "PsTD"
    df.columns.values[11] = "PsInt"
    df.columns.values[13] = "RshYds"
    df.columns.values[14] = "RshTD"
    df.columns.values[16] = "RecYds"
    df.columns.values[17] = "RecTD"
    df.fillna(0, inplace=True) # na occurs when a player has 0 years of experience

    cat_cols = ["Pos", "Player", "Tm", "All-pro teams"] # not converted to ints
    num_cols = [col for col in df.columns if col not in cat_cols]
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df["Year"] = yr
    
    return df
        

In [3]:
yrs = list(range(2000,2025))
df_list = []
for year in yrs:
    yr_df = year_scrape(year)
    df_list.append(yr_df)

all_pro_df = pd.concat(df_list, ignore_index=True)

all_pro_df

,Pos,Player,Tm,Age,Yrs,G,GS,Cmp,PsAtt,PsYds,...,RshYds,RshTD,Rec,RecYds,RecTD,Solo,Sk,Int,All-pro teams,Year
0,QB,Peyton Manning,IND,24,2.0,16,16,357,571,4413,...,116,1,0,0,0,NaN,0.0,0,AP: 2nd Tm,2000
1,TE,Frank Wycheck,TEN,29,7.0,16,16,2,2,53,...,0,0,70,636,4,0.0,0.0,0,AP: 2nd Tm,2000
2,T,Kyle Turley,NOR,25,2.0,16,16,0,0,0,...,0,0,1,16,0,NaN,0.0,0,AP: 1st Tm,2000
3,G,Ruben Brown,BUF,28,5.0,16,16,0,0,0,...,0,0,0,0,0,0.0,0.0,0,AP: 2nd Tm,2000
4,C,Kevin Mawae,NYJ,29,6.0,16,16,0,0,0,...,0,0,0,0,0,0.0,0.0,0,"AP: 2nd Tm, FW: 1st Tm",2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,KR,Derius Davis,LAC,24,1.0,15,2,0,0,0,...,39,0,13,112,2,0.0,0.0,0,FW: 1st Tm All-Conf.,2024
2114,KR,Austin Ekeler,WAS,29,7.0,12,6,0,0,0,...,367,4,35,366,0,0.0,0.0,0,AP: 2nd Tm,2024
2115,ST,Brenden Schooler,NWE,27,2.0,17,0,0,0,0,...,0,0,0,0,0,9.0,2.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ...",2024
2116,ST,J.T. Gray,NOR,28,6.0,17,0,0,0,0,...,0,0,0,0,0,12.0,0.0,0,"FW: 1st Tm All-Conf., AP: 2nd Tm, PFF: 1st Tm",2024


#### clean up dataframe for positional analysis

In [6]:
all_pro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2118 entries, 0 to 2117
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pos            2118 non-null   object 
 1   Player         2118 non-null   object 
 2   Tm             2118 non-null   object 
 3   Age            2118 non-null   int64  
 4   Yrs            2017 non-null   float64
 5   G              2118 non-null   int64  
 6   GS             2118 non-null   int64  
 7   Cmp            2118 non-null   int64  
 8   PsAtt          2118 non-null   int64  
 9   PsYds          2118 non-null   int64  
 10  PsTD           2118 non-null   int64  
 11  PsInt          2118 non-null   int64  
 12  Att            2118 non-null   int64  
 13  RshYds         2118 non-null   int64  
 14  RshTD          2118 non-null   int64  
 15  Rec            2118 non-null   int64  
 16  RecYds         2118 non-null   int64  
 17  RecTD          2118 non-null   int64  
 18  Solo    

In [4]:
all_pro_df['Pos'].unique()

array(['QB', 'TE', 'T', 'G', 'C', 'DE', 'DT', 'OLB', 'MLB', 'CB', 'S',
       'P', 'RDE', 'RCB', 'WR', 'LG', 'LT', 'K', 'FS', 'RLB', 'RDT', 'SS',
       'RB', 'LOLB', 'LDT', 'LCB', 'DB', 'RG', 'LLB', 'LB', 'ILB', 'LDE',
       'KR', 'ROLB', 'RT', 'RILB', 'LT/LG', 'FB', 'LILB', 'NT', 'LDT/RDT',
       'PR', 'ST', 'PK', 'RET', 'LS', 'EDGE', 'OT', 'FLEX', 'DL',
       'FLEX-O', 'FLEX-D'], dtype=object)

In [5]:
# clean up position column
# dictionary to group similar positions together
pos_dict = {
    # tackles
    'RT': 'T', 'LT': 'T', 'OT': 'T',
    # guards
    'RG': 'G', 'LG': 'G', 'LT/LG': 'G',
    # DT
    'RDT': 'DT', 'LDT': 'DT', 'NT': 'DT', 'LDT/RDT': 'DT','DL': 'DT', 
    # DE/edge
    'DE': 'EDGE', 'OLB': 'EDGE', 'RDE': 'EDGE', 'LDE': 'EDGE', 'LOLB': 'EDGE', 'ROLB': 'EDGE',
    # linebackers
    'MLB': 'LB', 'ILB': 'LB', 'LLB': 'LB', 'RLB': 'LB', 'RILB': 'LB', 'LILB': 'LB',
    # safety
    'FS': 'S', 'SS': 'S', 'FLEX-D': 'S',
    # cornerback
    'DB': 'CB', 'RCB': 'CB', 'LCB': 'CB',
    # kickers
    'PK': 'K',
    # returner
    'KR': 'RET', 'PR': 'RET',
    # cordarrelle patterson 
    'FLEX-O': 'RB'
}
# replace to common value in dataframe
all_pro_df['Pos'] = all_pro_df['Pos'].replace(pos_dict)
# special cases where player's position was FLEX
all_pro_df.loc[(all_pro_df['Player'] == 'Christian McCaffrey') & (all_pro_df['Year']==2018), 'Pos'] = 'RB'
all_pro_df.loc[(all_pro_df['Player'] == 'Alvin Kamara') & (all_pro_df['Year']==2018), 'Pos'] = 'RB'
all_pro_df.loc[(all_pro_df['Player'] == 'Austin Ekeler') & (all_pro_df['Year']==2019), 'Pos'] = 'RB'
all_pro_df.loc[(all_pro_df['Player'] == 'Allen Robinson') & (all_pro_df['Year']==2020), 'Pos'] = 'WR'
all_pro_df.loc[(all_pro_df['Player'] == 'Puka Nacua') & (all_pro_df['Year']==2024), 'Pos'] = 'WR'
all_pro_df.loc[(all_pro_df['Player'] == 'Drake London') & (all_pro_df['Year']==2024), 'Pos'] = 'WR'

In [6]:
all_pro_df['Pos'].unique()

array(['QB', 'TE', 'T', 'G', 'C', 'EDGE', 'DT', 'LB', 'CB', 'S', 'P',
       'WR', 'K', 'RB', 'RET', 'FB', 'ST', 'LS'], dtype=object)

In [7]:
all_pro_df.to_csv('all_pro_data.csv')

In [2]:
# df = all_pro_df
df = pd.read_csv('all_pro_data.csv')
round(df.groupby('Pos')['Age'].describe().sort_values(by='25%'), 3)

,count,mean,std,min,25%,50%,75%,max
Pos,,,,,,,,
RET,96.0,25.583,3.029,21.0,23.0,25.0,28.00,33.0
CB,176.0,26.693,3.066,21.0,24.0,26.0,29.00,35.0
RB,135.0,25.615,2.489,21.0,24.0,25.0,27.00,34.0
ST,83.0,26.867,3.196,21.0,24.5,26.0,29.00,36.0
S,160.0,27.125,3.209,21.0,25.0,27.0,29.00,39.0
DT,152.0,27.303,2.812,22.0,25.0,27.0,29.00,35.0
EDGE,248.0,27.185,2.852,21.0,25.0,27.0,29.00,35.0
LB,172.0,27.244,3.307,21.0,25.0,27.0,29.00,37.0
WR,175.0,26.834,3.025,21.0,25.0,27.0,29.00,40.0


In [3]:
# want to sort in order of 25% values
pos_order = (
    df.groupby('Pos')['Age']
    .quantile(0.5)
    .sort_values()
    .index
)

fig = px.box(
    df,
    x='Pos',
    y='Age',
    title='Age Distribution by Position',
    hover_data=['Player', 'Year'],  # Add extra info to hover tooltip
    category_orders={'Pos': pos_order}  # enforces order on the x-axis
)

fig.show()


In [4]:

# choose only 16 positions for 4x4 subplits
df1 = df[~df['Pos'].isin(['LS', 'P'])]

# remaining unique positions
positions = sorted(df1['Pos'].unique())

# create 4x4 subplots
rows, cols = 4, 4
fig = make_subplots(rows=rows, cols=cols, subplot_titles=positions)

# add histograms
for i, pos in enumerate(positions):
    row = i // cols + 1
    col = i % cols + 1
    subset = df1[df1['Pos'] == pos]
    
    fig.add_trace(
        go.Histogram(
            x=subset['Age'],
            xbins=dict(start=21, end=44, size=2),  # Set bin size and range
            name=pos,
            showlegend=False
        ),
        row=row,
        col=col
    )

# update layout and axis ranges
fig.update_layout(
    height=900,
    width=900,
    title_text="Age Distribution by Position (Excluding ST and P)",
    margin=dict(t=80),
    bargap=0.1
)

# set x-axis range for all subplots
for i in range(1, rows * cols + 1):
    fig.update_xaxes(range=[21, 44], row=(i - 1) // cols + 1, col=(i - 1) % cols + 1)
    fig.update_yaxes(title_text="Count", row=(i - 1) // cols + 1, col=(i - 1) % cols + 1)

fig.show()

##### takeaways
- Overall defense younger compared to WR TE
- RBs very young
- OL older than DL, Centers slightly higher than G and T

In [5]:
# want to sort in order of 25% values
pos_order = (
    df.groupby('Pos')['Yrs']
    .quantile(0.5)
    .sort_values()
    .index
)

fig = px.box(
    df,
    x='Pos',
    y='Yrs',
    title='Yrs of Experience Distribution by Position',
    hover_data=['Player', 'Year'],  # Add extra info to hover tooltip
    category_orders={'Pos': pos_order}  # enforces order on the x-axis
)

fig.show()

In [10]:
# create 4x4 subplots
rows, cols = 4, 4
fig = make_subplots(rows=rows, cols=cols, subplot_titles=positions)

# add histograms
for i, pos in enumerate(positions):
    row = i // cols + 1
    col = i % cols + 1
    subset = df1[df1['Pos'] == pos]
    
    fig.add_trace(
        go.Histogram(
            x=subset['Yrs'],
            xbins=dict(start=0, end=22, size=2),  # Set bin size and range
            name=pos,
            showlegend=False
        ),
        row=row,
        col=col
    )

# update layout and axis ranges
fig.update_layout(
    height=900,
    width=900,
    title_text="Yrs of experience Distribution by Position",
    margin=dict(t=80),
    bargap=0.1
)

# set x-axis range for all subplots
for i in range(1, rows * cols + 1):
    fig.update_xaxes(range=[0, 22], row=(i - 1) // cols + 1, col=(i - 1) % cols + 1)
    fig.update_yaxes(title_text="Count", row=(i - 1) // cols + 1, col=(i - 1) % cols + 1)

fig.show()

In [8]:
fig = make_subplots(rows=rows, cols=cols, subplot_titles=positions)

# Add scatterplots
for i, pos in enumerate(positions):
    row = i // cols + 1
    col = i % cols + 1
    subset = df1[df1['Pos'] == pos]

    fig.add_trace(
        go.Scatter(
            x=subset['Age'],
            y=subset['Yrs'],
            mode='markers',
            marker=dict(size=6, opacity=0.7),
            name=pos,
            hovertemplate=(
                "Player: %{customdata[0]}<br>"
                "Year: %{customdata[1]}<br>"
                "Age: %{x}<br>"
                "Yrs: %{y}<extra></extra>"
            ),
            customdata=subset[['Player', 'Year']].values
        ),
        row=row,
        col=col
    )

# Layout
fig.update_layout(
    height=900,
    width=900,
    title_text="Age vs. Years of Experience by Position",
    margin=dict(t=80),
    showlegend=False
)

# Show plot
fig.show()